In [ ]:
from keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils
from keras.models import Sequential
from keras.utils import array_to_img, img_to_array, load_img
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import pandas as pd
import numpy as np### 데이터 준비 완료 -----------

from sklearn.metrics import accuracy_score

In [8]:
path = r"C:\\Users\\USER\\Desktop\\font_original"           #원본 이미지 경로
copy_path = r"C:\\Users\\USER\\Desktop\\font_classifier_7.1"    #train/test/val 폴더로 분류한 이미지 경로

In [9]:
#train/test 폴더로 분류
import splitfolders

splitfolders.ratio(path, output=copy_path, seed = 43, ratio = (.7,.3))

Copying files: 55860 files [04:22, 213.02 files/s]


In [11]:
trainFileList = os.listdir(copy_path+'\\train')                    #train 폴더 경로
testFileList = os.listdir(copy_path+'\\test')                       #val 폴더 경로

In [12]:
trainFileList = [x for x in trainFileList if 'jpg' in x]
testFileList = [x for x in testFileList if 'jpg' in x]

In [13]:
trainDataGen = ImageDataGenerator(rescale=1./255)

In [14]:
trainGenSet = trainDataGen.flow_from_directory(
    copy_path + '\\train',
    batch_size=64,
    target_size=(28,28),
    class_mode='categorical'
)

Found 39100 images belonging to 5 classes.


In [15]:
testDataGen = ImageDataGenerator(rescale=1./255)

In [16]:
testGenSet = testDataGen.flow_from_directory(
    copy_path + '\\test',
    target_size=(28,28),
    batch_size=64,
    class_mode='categorical'
)

Found 16760 images belonging to 5 classes.


In [17]:
valDataGen = ImageDataGenerator(rescale=1./255,
                                 rotation_range = 15,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 shear_range=0.2,
                                 zoom_range=0.1,
                                 horizontal_flip=False,
                                 vertical_flip=False,
                                 fill_mode='nearest')

valGenSet = valDataGen.flow_from_directory(
    copy_path + '\\train',
    target_size=(28,28),
    batch_size=64,
    class_mode='categorical'
)

Found 39100 images belonging to 5 classes.


In [18]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), padding='same', input_shape=(28,28,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, kernel_size=(3,3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(18, activation='softmax'))

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0

In [20]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [21]:
model.fit_generator(
    trainGenSet,
    steps_per_epoch=40,
    epochs=400,
    validation_data=valGenSet,
    validation_steps=20,
)

C:\Users\USER\AppData\Local\Temp\ipykernel_5260\64282257.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/400


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "C:\Use